In [1]:
#Objective Classify spam text message

import numpy as np 
import pandas as pd 
import nltk as nlp
import keras.optimizers
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Model
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop,Adam, SGD, Adadelta
#from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
# load data
data = pd.read_csv("/content/SPAM text message 20170820 - Data.csv")

In [3]:
data.shape


(5572, 2)

In [4]:
# convert spam to 1, not spam to 0
data["Category"] = [1 if category == "spam" else 0 for category in data["Category"]]

In [5]:
from sklearn.preprocessing import LabelEncoder
X = data.Message
Y = data.Category
le = LabelEncoder()
Y = le.fit_transform(Y)
print(Y.shape)
Y = Y.reshape(-1,1)
Y=Y.reshape(-1)
print(Y.shape)

(5572,)
(5572,)


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(4736,)
(836,)
(4736,)
(836,)


In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
print(sequences_matrix.shape)

(4736, 150)


In [80]:
def RNN1():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = SimpleRNN(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dense(128,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [81]:
model = RNN1()
#model1=RNN1()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(), metrics=['accuracy'])
#optimizer=RMSprop

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding_10 (Embedding)    (None, 150, 50)           50000     
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 128)               22912     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation_30 (Activation)  (None, 256)               0         
                                                                 
 FC2 (Dense)                 (None, 128)               32896     
                                                                 
 activation_31 (Activation)  (None, 128)               0  

In [75]:
model.fit(sequences_matrix,Y_train,batch_size=16,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
237/237 [==============================] - 29s 112ms/step - loss: 0.1354 - accuracy: 0.9533 - val_loss: 0.0733 - val_accuracy: 0.9810
Epoch 2/10
237/237 [==============================] - 25s 106ms/step - loss: 0.0454 - accuracy: 0.9879 - val_loss: 0.0693 - val_accuracy: 0.9810


In [76]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
test_sequences_matrix.shape
print(Y_test.shape)


(836,)


In [77]:
accr = model.evaluate(test_sequences_matrix,Y_test)
print(accr)

27/27 [==============================] - 1s 30ms/step - loss: 0.0967 - accuracy: 0.9761
[0.09673529118299484, 0.9760765433311462]


In [79]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.097
  Accuracy: 0.976


In [ ]:
# Defining the arguments that we want to use in Grid Search along  
# with the list of values that we want to try out  
from sklearn.model_selection import GridSearchCV, KFold  
learnRate = [0.001, 0.02, 0.2]  
dropoutRate = [0.0, 0.2, 0.4]  
batchSize = [10, 20, 30]  
epochs = [1, 5, 10]  
  
model = Sequential()
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = GRU(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
# Making a dictionary of the grid search parameters  
paramgrid = dict(learning_rate = learnRate, Dropout = dropoutRate, batch_size = batchSize, epochs = epochs )  
#model = KerasClassifier(model=RNN(), epochs=10)
# Building and fitting the GridSearchCV  
mygrid = GridSearchCV(estimator=model, param_grid = paramgrid, scoring='accuracy', cv = KFold(random_state = None), verbose = 0)  
#model = KerasClassifier(model, verbose=0)
gridresults = mygrid.fit(sequences_matrix,Y_train)  
  
# Summarizing the results in a readable format  
print("Best: " + gridresults.best_score_ + " using " + gridresults.best_params_)  
  
means = gridresults.cv_results_['mean_test_score']  
stds = gridresults.cv_results_['std_test_score']  
params = gridresults.cv_results_['params']  
  
for mean, stdev, param in zip(means, stds, params):  
    print(mean + "(" + stdev + ")" + " with: " + param)  

TypeError: ignored